# Vérification et renommage des colonnes

In [1]:
import pandas as pd

df = pd.read_csv("../data/raw/MatchDetailStats_20222023.csv", sep=";", encoding="latin1")

# Suppression des espaces autour des noms de colonnes
df.columns = df.columns.str.strip()

# Remplacement des accents et passage en minuscules
df.columns = (
    df.columns
    .str.replace('é','e')
    .str.replace('è','e')
    .str.replace('à','a')
    .str.replace('î','i')
    .str.lower()
)
df.columns


Index(['matchid', 'journee', 'stade', 'spectateurs', 'date', 'home_team',
       'away_team', 'home_goal', 'away_goal', 'home_possession',
       'away_possession', 'home_duels_gagnes', 'away_duel_gagnes',
       'home_duel_aeriens_gagnes', 'away_duel_aeriens_gagnes',
       'home_interceptions', 'away_interceptions', 'home_hors_jeux',
       'away_hors_eux', 'home_corners_won', 'away_corners_won', 'home_passes',
       'away_passes', 'home_centres', 'away_centres', 'home_tirs', 'away_tirs',
       'home_tirs_cadres', 'away_tirs_cadres', 'home_carton_jaune',
       'away_carton_jaune', 'home_carton_rouge', 'away_carton_rouge',
       'home_formation', 'away_formation'],
      dtype='object')

# Conversion des types

In [2]:
# Colonnes à convertir en entier
cols_int = ['home_goal','away_goal','home_tirs','away_tirs','home_tirs_cadres','away_tirs_cadres']

# Remplacer les NaN par 0 puis convertir en int
df[cols_int] = df[cols_int].fillna(0).astype(int)

# Vérifier
df[cols_int].isna().sum()
df[cols_int].dtypes


home_goal           int64
away_goal           int64
home_tirs           int64
away_tirs           int64
home_tirs_cadres    int64
away_tirs_cadres    int64
dtype: object

In [3]:
# Colonnes numériques avec virgules
cols_percentage = ['home_possession','away_possession']
for col in cols_percentage:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',','.'), errors='coerce')

# Colonnes entières
cols_int = ['home_goal','away_goal','home_tirs','away_tirs','home_tirs_cadres','away_tirs_cadres']
df[cols_int] = df[cols_int].fillna(0).astype(int)

# Forcer en string
df['date_str'] = df['date'].astype(str)

# Nettoyer les jours de la semaine
df['date_str'] = df['date_str'].str.replace(r'DIM\. |LUN\. |MAR\. |MER\. |JEU\. |VEN\. |SAM\. ', '', regex=True)

# Nettoyer le séparateur de l'heure
df['date_str'] = df['date_str'].str.replace(' - ', ' ', regex=False)

# Convertir en datetime
df['date'] = pd.to_datetime(df['date_str'], format='%d %B %Y %H:%M', errors='coerce')


# Colonnes supplémentaires

In [4]:
# Résultat du match pour l'équipe à domicile
df['home_win'] = (df['home_goal'] > df['away_goal']).astype(int)

# Total de tirs
df['total_tirs'] = df['home_tirs'] + df['away_tirs']

# Différence de possession
df['diff_possession'] = df['home_possession'] - df['away_possession']


# Export du CSV clean

In [5]:
df.to_csv("../data/raw/ligue1_clean.csv", index=False, sep=";")